In [0]:
#Install Pytorch
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# workaround 
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

In [0]:
import torch

#GPU (bzw. alternativ CPU) referenzieren
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(4, 4)    
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 70)
        self.fc3 = nn.Linear(70, 5)
        #Dropout-Schicht definieren
        self.dropout = nn.Dropout(p=0.05)
    
    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        #Dropout-Schicht verwenden
        x = F.relu(self.fc2(self.dropout(x)))
        #Softmax-Aktivierungsfunktion statt ReLU auf Ausgabe-Schicht anwenden (Für Fehlerfunktion NLLLoss empfohlen)
        x = F.log_softmax(self.fc3(x))
        return x   

def train(net, epochs):    
    net.to(device)    
    trainloader = load_dataset('drive/My Drive/pytorch_google_imgs/train')
    #Fehlerfunktion tauschen
    criterion = nn.NLLLoss() #nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    
    start = time.time()
    for epoch in range(epochs): 
        for i, data in enumerate(trainloader, 0):            
            inputs, labels = data            
            inputs, labels = inputs.to(device), labels.to(device)
           
            optimizer.zero_grad()        
            
            outputs = net(inputs)            
            
            loss = criterion(outputs, labels)
            loss.backward()            
           
            optimizer.step()
    
    end = time.time()
    timediff = (end-start)
    print('Computation time: '+str(timediff)+"s")
            
##############################################################################
# An den untenstehenden Hilfsmethoden müssen Sie keine Anpassungen vornehmen.#
##############################################################################
    
def getAccuracy(net):    
    net.to(device)
    testloader = load_dataset('drive/My Drive/pytorch_google_imgs/test')
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in testloader:
            images, labels = data            
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(str(correct)+" of the "+str(total)+" test images were classified correct ("+str(100 * correct / total)+"%).")
    
def showRandomPredictions(net):    
    net.to(device)
    testloader = load_dataset('drive/My Drive/pytorch_google_imgs/test')
    dataiter = iter(testloader)
    images, labels = dataiter.next()    
    images, labels = images.to(device), labels.to(device)
    
    #Zuordnung der Klassen erfolgt durch torch automatisch über den Ordnernamen in dieser Reihenfolge
    classes = ('intestines', 'kidneys', 'liver', 'lungs', 'stomach')
    
    #get predicted results
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    
    # print images
    imshow(torchvision.utils.make_grid(images))
    print('Correct classes are:  ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))   
    print('Classifier predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))
    

def load_dataset(data_dir):
    transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])        
    dset = torchvision.datasets.ImageFolder(data_dir, transform)     
    return torch.utils.data.DataLoader(dset, batch_size=4, shuffle=True, num_workers=4)

def imshow(img):
    img = img / 2 + 0.5    
    img = img.cpu()
    npimg = img.numpy()    
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

#Ausschließlich die Parameter des Netzes (net.state_dict()) auf Drive unter dem Namen 'HandsOn11Params' speichern
def saveParams(net):    
    torch.save(net.state_dict(), 'drive/My Drive/pytorch_google_imgs/HandsOn11Params')

#Dem Netz die gespeicherten Parameter aus Drive laden    
def loadParams(net):
    net = net.load_state_dict(torch.load('drive/My Drive/pytorch_google_imgs/HandsOn11Params'))

#In Windows muss aufgrund eines Bugs diese if-Abfrage gesetzt sein
if __name__ == "__main__":
    __spec__ = None    
    net = MyClassifier()
    
    #Parameter laden oder neu trainieren
    load = False
    if load:        
        loadParams(net)
    else:        
        train(net, 2)
        
    getAccuracy(net)
    showRandomPredictions(net)
    
    #Netz abspeichern
    saveParams(net)